In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [57]:
import re
from glob import glob
import os
import pandas
import sklearn.metrics
import collections

In [49]:
def find_peak(result, scoring_function):
    keys = sorted(result[0].keys())
    guesses = [concatenate([fold[key]["guesses"] for fold in result]) for key in keys]
    true_labels = concatenate([fold[keys[0]]["true_labels"] for fold in result])
    scores = [scoring_function(true_labels, guess) for guess in guesses]
    return keys[argmax(scores)]

In [44]:
class0_recall = lambda gt, pred: sklearn.metrics.recall_score(gt, pred, pos_label=0)
class1_recall = lambda gt, pred: sklearn.metrics.recall_score(gt, pred, pos_label=1)

In [51]:
def calculate_scores(result, key):
    guess = concatenate([fold[key]["guesses"] for fold in result])
    true_labels = concatenate([fold[key]["true_labels"] for fold in result])
    return {
        "accuracy": sklearn.metrics.accuracy_score(true_labels, guess),
        "class0_recall": class0_recall(true_labels, guess),
        "class1_recall": class1_recall(true_labels, guess),
    }

In [87]:
run = "adftld_0_0"
pattern = re.compile(r"^.*_tfphammers$")
results = {}

In [88]:
filenames = [fn for fn in glob("results/{}_*.csv".format(run))
             if pattern.match(os.path.basename(fn)[len(run) + 1:-4])]
len(filenames)

57

In [89]:
for filename in filenames:
    df = pandas.read_csv(filename, dtype={"labels": str})
    data = []
    for fold in df.fold.unique():
        fold_df = df[df.fold == fold]
        guess_df = fold_df[fold_df.param > 0].sort_values("param")
        ground_truth = [int(c) for c in fold_df[fold_df.param == 0]["labels"].as_matrix()[0]]
        guesses = guess_df["labels"].as_matrix()
        params = guess_df["param"].as_matrix()
        narcs = guess_df[["narcs0", "narcs1"]].as_matrix()
        fold_data = {}
        for param, guess, narcs in zip(params, guesses, narcs):
            fold_data[param] = {"true_labels": ground_truth, "guesses": [int(c) for c in guess],
                                "narcs": narcs}
        data.append(fold_data)
    method, _ = os.path.splitext(filename)
    method = method.split("_", 3)[3].replace("_compact", "")
    results[method] = data

In [90]:
peak_accuracies = {k: find_peak(v, sklearn.metrics.accuracy_score) for k, v in results.items()}
peak_class0_recalls = {k: find_peak(v, class0_recall) for k, v in results.items()}
peak_class1_recalls = {k: find_peak(v, class1_recall) for k, v in results.items()}

In [91]:
method_groups = collections.defaultdict(list)
for method in results.keys():
    _, method_group = method.split("_", 1)
    method_groups[method_group].append(method)

In [92]:
for method_group, methods in method_groups.items():
    for m in methods:
        sc = calculate_scores(results[m], peak_accuracies[m])
        print("{:<50}    {:.2f}    {:.2f}    {:.2f}".format(m, sc["accuracy"], sc["class0_recall"], sc["class1_recall"]))
    print()

exp0.05_fa_tfphammers                                 0.79    0.83    0.73
pow3_fa_tfphammers                                    0.89    0.90    0.87
pow1_fa_tfphammers                                    0.83    0.85    0.80
exp0.4_fa_tfphammers                                  0.86    0.85    0.87
pow2_fa_tfphammers                                    0.89    0.90    0.87
exp0.1_fa_tfphammers                                  0.83    0.85    0.80
exp1_fa_tfphammers                                    0.83    0.85    0.80
exp0.8_fa_tfphammers                                  0.83    0.85    0.80
exp0.2_fa_tfphammers                                  0.90    0.93    0.87
exp0.01_fa_tfphammers                                 0.79    0.80    0.77
exp0.02_fa_tfphammers                                 0.79    0.83    0.73

pow3_tfphammers                                       0.85    0.85    0.83
pow1_tfphammers                                       0.83    0.85    0.80
exp1_tfphammers         